<a href="https://colab.research.google.com/github/smilkes/AR-Bee-Project/blob/main/First_Order_Motion_Model_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook contains some code to test [the paper First Order Motion Model for Image Animation by Aliaksandr Siarohin, Stéphane Lathuilière, Sergey Tulyakov, Elisa Ricci and Nicu Sebe. ](https://github.com/AliaksandrSiarohin/first-order-model)

# Usage

You can run all the cells to get a sense of what it can do.
Then check the configuration cell to try with your own data

# First Step
The next cell will download the default data, the paper code, and the model checkpoint

In [1]:
!wget -O 'vox-cpk.tar' https://www.dropbox.com/s/fgmmcelslthqbdu/vox-cpk.pth.tar?dl=0
!wget -O 'default_img.png' 'https://www.dropbox.com/s/jerr2li69j6sis6/jon_snow.png?dl=0'
!wget -O 'default_video.mp4' 'https://www.dropbox.com/s/zrq6wtnyq1idvsb/04.mp4?dl=0'
!git clone https://github.com/AliaksandrSiarohin/first-order-model

--2021-11-23 04:37:28--  https://www.dropbox.com/s/fgmmcelslthqbdu/vox-cpk.pth.tar?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fgmmcelslthqbdu/vox-cpk.pth.tar [following]
--2021-11-23 04:37:28--  https://www.dropbox.com/s/raw/fgmmcelslthqbdu/vox-cpk.pth.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 404 Not Found
2021-11-23 04:37:28 ERROR 404: Not Found.

--2021-11-23 04:37:28--  https://www.dropbox.com/s/jerr2li69j6sis6/jon_snow.png?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/jerr2li69j6sis6/jon_snow.png [following]
--2021-11-

In [2]:
cd first-order-model/

/content/first-order-model


# Some code

Some code to interact with the model

In [3]:
import os
from demo import *
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def generate_video(file_name, start, end, w, h, x, y, delete_last_video):
    if delete_last_video:
      os.system('rm out.mp4')
    os.system('ffmpeg -i %s -ss %s -t %s -filter:v "crop=%s:%s:%s:%s" -async 1 out.mp4' % (file_name, start, end, w, h, x, y))

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

def generate_result(image_name):
  source_image = imageio.imread(image_name)
  driving_video = imageio.mimread('out.mp4', memtest=False)


  #Resize image and video to 256x256

  source_image = resize(source_image, (256, 256))[..., :3]
  driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

  predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                              adapt_movement_scale=True)

  anim =  display(source_image, driving_video, predictions)
  Writer = animation.writers['ffmpeg']
  writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
  anim.save('animation.mp4', writer=writer)

 

# Configuration

You can play with the next configuration params:


*   **video_start** - set left cut of the video
*   **video_end** - set right cut of the video
*   **video_width** - set the selected video width box
*   **video_height** - set the selected video height box
*   **video_x** - set where the video box starts in x axis
*   **video_y** - set where the video box starts in y axis
*   **video_name** - name of your own video file
*   **img_name** - name of your own image file
*   **use_default_img** - use default image instead of your own
*   **use_default_video** - use default video instead of your own
*   **delete_last_video** - don't delete parsed video, usefull if you are going to play with 1 video and changing images.


In [4]:
########Params###################
video_start = '00:00:00.50'
video_end = '00:00:07'
video_width = 256
video_height = 256
video_x = 0
video_y = 0
video_name = "video.mp4"
img_name = 'img.png'
use_default_img = True
use_default_video = True
delete_last_video = True
#########Params##################
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='../vox-cpk.tar')
if use_default_img:
  img_name = "default_img.png"
if use_default_video:
  video_name = "default_video.mp4"

EOFError: ignored

# Where the real magic happens

In [ ]:
# Run this every time you change video
generate_video('../%s' % video_name, video_start, video_end, video_width, video_height, video_x, video_y, delete_last_video)

In [ ]:
# Run this every time 
generate_result('../%s' % img_name)

100%|██████████| 196/196 [00:07<00:00, 27.07it/s]


In [ ]:
from base64 import b64encode
mp4 = open('animation.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1200 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Your turn

Upload an image and a video, play with the parameters, then run the cells from the above cells.

If you want to download the video, its in the left, in first-order-model, a file called animation.mp4